In [ ]:
import k3d
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
from bvhtoolbox import BvhTree
import transforms3d as t3d
from pyergonomics.importers import Unit

#UNIT = Unit.CM
#BVH_FILE = "../examples/data/xsens/Case_Study_12-001.bvh"

UNIT = Unit.M
BVH_FILE = "../examples/data/optitrack/umu_002.bvh"


with open(BVH_FILE) as f:
    bvh = BvhTree(f.read())

joints = list(bvh.get_joints())
joint_names = [j.name for j in joints]
print(f"Joints: {joint_names}")

def get_positions(frame):
    """Compute FK for a single frame."""
    positions = []
    transforms = {}  # joint_name -> 4x4 matrix
    
    for joint in joints:
        channels = bvh.joint_channels(joint.name)
        values = bvh.frame_joint_channels(frame, joint.name, channels)
        
        # Parse channels
        pos = np.zeros(3)
        rot = np.eye(3)
        for ch, val in zip(channels, values):
            rad = np.radians(val)
            if ch == 'Xposition': pos[0] = val
            elif ch == 'Yposition': pos[1] = val
            elif ch == 'Zposition': pos[2] = val
            elif ch == 'Xrotation': rot = rot @ t3d.euler.euler2mat(rad, 0, 0, 'sxyz')
            elif ch == 'Yrotation': rot = rot @ t3d.euler.euler2mat(0, rad, 0, 'sxyz')
            elif ch == 'Zrotation': rot = rot @ t3d.euler.euler2mat(0, 0, rad, 'sxyz')
        
        local = t3d.affines.compose(pos, rot, np.ones(3))
        
        # Apply offset and parent transform
        parent = joint.parent
        parent_name = parent.value[1] if parent and len(parent.value) > 1 else None
        if parent_name and parent_name in transforms:
            offset = [float(o) for o in joint['OFFSET']]
            local[:3, 3] = offset
            world = transforms[parent_name] @ local
        else:
            world = local
        
        transforms[joint.name] = world
        
        # Convert Y-up to Z-up: (x, y, z) -> (x, -z, y), and scale
        p = world[:3, 3] * UNIT.value
        positions.append([p[0], -p[2], p[1]])
    
    return positions

# Plot
plot = k3d.plot(camera_mode='orbit')
joints_plot = k3d.points(get_positions(0), point_size=0.03, color=0xff0000)
plot += joints_plot
plot.display()

def set_frame(n):
    joints_plot.positions = get_positions(n)

interact(set_frame, n=widgets.IntSlider(min=0, max=bvh.nframes-1, value=0, layout=widgets.Layout(width='100%')))

In [ ]:
# Compare with pyergonomics import
from pyergonomics.importers import from_bvh

settings = from_bvh(BVH_FILE, unit=UNIT)
tracker = settings.tracker

def get_positions_pyergo(frame):
    return tracker.get_keypoints_for_person(1, frame=frame)[0]

# Plot both: red = notebook FK, blue = pyergonomics
plot2 = k3d.plot(camera_mode='orbit')
notebook_plot = k3d.points(get_positions(0), point_size=0.03, color=0xff0000)  # red
pyergo_plot = k3d.points(get_positions_pyergo(0), point_size=0.03, color=0x0000ff)  # blue
plot2 += notebook_plot
plot2 += pyergo_plot
plot2.display()

def set_frame2(n):
    notebook_plot.positions = get_positions(n)
    pyergo_plot.positions = get_positions_pyergo(n)

interact(set_frame2, n=widgets.IntSlider(min=0, max=bvh.nframes-1, value=0, layout=widgets.Layout(width='100%')))